# Taking advantage of Colab Pro



## Faster GPUs

With Colab Pro you have priority access to our fastest GPUs. For example, you may get a T4 or P100 GPU at times when most users of standard Colab receive a slower K80 GPU. You can see what GPU you've been assigned at any time by executing the following cell.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Aug 10 03:10:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In order to use a GPU with your notebook, select the Runtime > Change runtime type menu, and then set the hardware accelerator dropdown to GPU.

## More memory

With Colab Pro you have the option to access high-memory VMs when they are available. To set your notebook preference to use a high-memory runtime, select the Runtime > 'Change runtime type' menu, and then select High-RAM in the Runtime shape dropdown.

You can see how much memory you have available at any time by running the following code.


In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import numpy as np
from sklearn import svm, metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow import keras
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array, array_to_img
from skimage.transform import resize, rescale
import matplotlib.pyplot as plt
from skimage import data, color
from PIL import Image
import cv2
from numpy import savez_compressed
from google.colab import files
from numpy import load
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
from google.colab import drive
drive.mount('/content/gdrive')

filename = '/content/gdrive/My Drive/Research2/np_data/x.npz'
x = np.load(filename)['arr_0']

filename = '/content/gdrive/My Drive/Research2/np_data/y.npz'
y = np.load(filename)['arr_0']

print(x.shape)
print(y.shape)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
(270912, 56, 56, 1)
(270912, 1)


## Longer runtimes

All Colab runtimes are reset after some period of time (which is faster if the runtime isn't executing code). While Colab Pro subscribers still have limits, these will be roughly twice the limits for non-subscribers.

In [ ]:
baseAccuracies = []
losses = []
balancedAccuracies = []
f1scores = []
precisions = []
recalls = []

kfold = KFold(n_splits=10, shuffle=True)
fold_no = 1
callback = EarlyStopping(monitor = 'val_loss', patience = 2, verbose = 1)

for train, test in kfold.split(x, y):
  print("_________________________________________________________________________________________________________________________________")
  print("Fold number: " + str(fold_no))
  fold_no = fold_no+1
  print()

  xtrain = x[train]
  xtest = x[test]
  
  model = tf.keras.applications.VGG16(include_top=True, weights=None, input_shape=(56, 56, 1), classes=49, classifier_activation="softmax",)
  model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(lr=0.0001), metrics=["sparse_categorical_accuracy"])
  model.fit(xtrain, y[train], verbose = 1, validation_data = (xtest, y[test]), epochs = 10, callbacks = [callback], batch_size = 32)

  test_loss, test_accuracy = model.evaluate(xtest, y[test])
  print("_________________________________________________________________________________________________________________________________")
  print("Test Accuracy:")
  print(test_accuracy)
  baseAccuracies.append(test_accuracy)
  print("_________________________________________________________________________________________________________________________________")
  print("Test Loss:")
  print(test_loss)
  losses.append(test_loss)
  print("_________________________________________________________________________________________________________________________________")
  preds = np.argmax(model.predict(xtest), axis=-1)
  preds = preds.reshape(len(xtest), 1)
  accs = []
  for cls in range(49):
    mask = (y[test] == cls)
    cls_acc = (preds == cls)[mask].mean() 
    accs.append(cls_acc)
  accs = np.mean(accs)
  print("Final balanced accuracy:")
  print(accs)
  balancedAccuracies.append(accs)
  print("_________________________________________________________________________________________________________________________________")
  f1scores.append(f1_score(y[test], preds, average = 'weighted'))
  precisions.append(precision_score(y[test], preds, average = 'weighted'))
  recalls.append(recall_score(y[test], preds, average = 'weighted'))
  print("*********************************************************************************************************************************")
  print()
  print()
  del model, xtrain, xtest

_________________________________________________________________________________________________________________________________
Fold number: 1

Epoch 1/10
7620/7620 [==============================] - 215s 28ms/step - loss: 0.5397 - sparse_categorical_accuracy: 0.8506 - val_loss: 0.2132 - val_sparse_categorical_accuracy: 0.9423
Epoch 2/10
7620/7620 [==============================] - 215s 28ms/step - loss: 0.1462 - sparse_categorical_accuracy: 0.9604 - val_loss: 0.1542 - val_sparse_categorical_accuracy: 0.9600
Epoch 3/10
7620/7620 [==============================] - 214s 28ms/step - loss: 0.1000 - sparse_categorical_accuracy: 0.9728 - val_loss: 0.1077 - val_sparse_categorical_accuracy: 0.9718
Epoch 4/10
7620/7620 [==============================] - 215s 28ms/step - loss: 0.0744 - sparse_categorical_accuracy: 0.9790 - val_loss: 0.1165 - val_sparse_categorical_accuracy: 0.9725
Epoch 5/10
7620/7620 [==============================] - 215s 28ms/step - loss: 0.0593 - sparse_categorical_accurac

In [ ]:
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(balancedAccuracies)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {losses[i]} - class-wise Accuracy: {balancedAccuracies[i]}% Accuracy - {baseAccuracies[i]} - Precision: {precisions[i]} - Recall: {recalls[i]} - F1: {f1scores[i]}')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(balancedAccuracies)} (+- {np.std(balancedAccuracies)})')
print(f'> Loss: {np.mean(losses)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.11145315319299698 - class-wise Accuracy: 0.9733667120640996% Accuracy - 0.9760814905166626 - Precision: 0.9762388450325744 - Recall: 0.9760815000738225 - F1: 0.9760372653784921
------------------------------------------------------------------------
> Fold 2 - Loss: 0.11910578608512878 - class-wise Accuracy: 0.9715321605237515% Accuracy - 0.9739406704902649 - Precision: 0.9742056417904237 - Recall: 0.9739406466853684 - F1: 0.9738678089594694
------------------------------------------------------------------------
> Fold 3 - Loss: 0.10233599692583084 - class-wise Accuracy: 0.9725513017660404% Accuracy - 0.9765235781669617 - Precision: 0.9766439748422074 - Recall: 0.9765235687128566 - F1: 0.9765030821539726
------------------------------------------------------------------------
> Fold 4 - Loss: 0.1088832095265

## Resource limits in Colab Pro

Your resources are not unlimited in Colab Pro. To make the most of Colab Pro, please avoid using resources when you don't need them. For example, only use a GPU or high-RAM runtime when required, and close Colab tabs when finished.


## Send us feedback!

If you have any feedback for us, please let us know. The best way to send feedback is by using the Help > 'Send feedback...' menu. If you encounter usage limits in Colab Pro and would be interested in a product with higher usage limits, do let us know.

If you encounter errors or other issues with billing (payments) for Colab Pro, please email colab-billing@google.com.